In [5]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 31: Likelihood Ratio Tests

Recall maximum likelihood estimators. These are obtained by maximizing the likelihood function with respect to $\theta$, the parameter of interest. Let's go through an example:

### Example 1: Poisson Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the Poisson distribution with unknown parameter $\lambda$. Find $\hat{\lambda}_{ML}$, the maximum likelihood estimate of $\lambda$. 

pdf: $\frac{e^{-\lambda} \lambda^x_i}{x_i !}$
Therefore,
$$ L(\lambda | x)=\prod_{i=1}^{n} \frac{e^{-\lambda} \lambda^{x_i}}{x_i !}        \quad \quad \quad\quad(1)$$
Take the log of (1),
$$ l(\lambda | x)=\sum_{i=1}^{n} ln(\frac{e^{-\lambda} \lambda^{x_i}}{x_i !})$$
Which can be simplified to:
$$ l(\lambda | x)=\sum_{i=1}^{n} ln(e^{-\lambda})+ln(\lambda^{x_i})-ln({x_i !})\quad \quad \quad\quad(2)$$
To find the "critical points", that is, the location of where the maximum occurs, take the derivative of (2) with respect to $\lambda$,
$$ \frac{\partial l(\lambda | x)}{\partial \lambda}=\sum_{i=1}^{n} \bigg{[}-1+\frac{x_i}{\lambda}\bigg{]}=0\quad \quad \quad\quad(3)$$
Now applying the sum as if being multiplied by $n$, we have,
$$-n+\frac{\sum_{i=1}^{n} x_i}{\lambda}=0\quad \quad \quad\quad(4)$$
Recall $\sum_{i=1}^{n} x_i=n\bar{x}$. Therefore, we conclude,
$$\lambda_{ML}=\bar{x}\quad \quad \quad\quad\quad \quad(5)$$

### Likelihood Ratio Tests

Assume you are testing a hypothesis:
$$
H_0: \theta=\theta_0
$$
$$
H_1: \theta\neq \theta_0
$$

The idea behind a likelihood ratio test is to compare the likelihood of the hypothesized value ($L(\theta_0\mid\textbf{x})$) to the maximum likelihood given the data ($L(\hat{\theta}_{ML}\mid\textbf{x})$). If the hypothesized value of $\theta$ were feasible, the likelihood under $\theta_0$ should be close to the max. If the hypothesized value of $\theta$ were not feasible, $L(\theta_0\mid\textbf{x})$ should be much smaller. To make the comparison, we consider the likelihood ratio test statistic, $\Lambda$: 

$$
\Lambda=\frac{L(\theta_0\mid\textbf{x})}{L(\hat{\theta}_{ML}\mid\textbf{x})}
$$

Because $\hat{\theta}_{ML}$ is the maximum likelihood estimator, this ratio has a maximum value of 1. Large values of $\Lambda$ (close to 1) indicate that $\theta_0$ is feasible (lack of evidence to reject). Small values of $\Lambda$ (close to 0) indicate $\theta_0$ is not feasible (evidence to reject). 

But how close to 0 is "close"? 

To evaluate this, we will take advantage of a helpful result. It turns out that if the null hypothesis were true, $-2\log \Lambda$ approximately follows the chi-squared distribution with 1 degree of freedom. The proof is outside the scope of this class. 

[We have not yet talked about the chi-squared distribution. To learn more, consult scipy help (`scipy.stats.chi2`). This distribution has one parameter that we care about: degrees of freedom, referenced in scipy as `df`. Bottom line, a random variable that has a chi-squared distribution with `df` degrees of freedom has a domain of $[0,\infty)$ and an expected value of `df`.]

$$
-2\log \Lambda = 2\left[l(\hat{\theta}_{ML}\mid \textbf{x})-l(\theta_0\mid \textbf{x})\right] \approx \textsf{Chisq}(1)
$$

It is a little bit more intuitive to consider $-2\log \Lambda$; large values of this test statistic indicate evidence to reject the null, which is consistent with most other hypothesis tests we've looked at. 

### Example 2: Likelihood ratio test on $\lambda$ from Poisson distribution

(Example taken from Pruim 2011). An instructor believes the the number of students who arrive late for class follows a Poisson process with an average of 1 late arrival per lesson. In 10 consecutive lessons, he found the following number of late arrivals: (1,1,0,4,2,1,3,0,0,2). Conduct a likelihood ratio test on the following hypothesis:

$$
H_0: \lambda = 1
$$
$$
H_1: \lambda \neq 1
$$

In [6]:
Data=[1,1,0,4,2,1,3,0,0,2]
lamb_ML=np.mean(Data) #Average of the data, and by example 1, is also the maximum likelihood of a possion distribution
lamb_ML

1.4

Before digging into the math, let's go over what we are going to calculate before actually doing it (so the coding makes sense.
Recall the likelihood test:
$$ L(\lambda | x)=\prod_{i=1}^{n} \frac{e^{-\lambda} \lambda^{x_i}}{x_i !} $$
And the ratio test:
$$
\Lambda=\frac{L(\theta_0\mid\textbf{x})}{L(\hat{\theta}_{ML}\mid\textbf{x})}
$$
As shown above, we have the average $\bar{x}=1.4$, which according to example 1, is our $\lambda_{ML}$. The numerator of the ratio test will be if $\lambda=1$.Therefore, the ratio that will be calculated is:
$$
\Lambda=\frac{L(1\mid\textbf{x})}{L(1.4\mid\textbf{x})}
$$

In [17]:
#Likelihood 1 is the hypothesized value:
def PoissonLikelihood(hyp_value,ActualData):
    Likelihood_hyp_value=np.prod([(e**-hyp_value)*(hyp_value**Value)/factorial(Value) for Value in Data])
    return Likelihood_hyp_value
Likelihood_hypothesizedValue=PoissonLikelihood(1,Data)
MaximumLikelihood=PoissonLikelihood(np.mean(Data),Data)
Cap_Lambda=Likelihood_hypothesizedValue/MaximumLikelihood
print(Cap_Lambda)

0.491343740966452


Ok, so we have a $\Lambda=0.491343740966452$. As mentioned above, this isn't close enough to zero to reject the null hypothesis, but what is? Since we clearly have no idea and hit the "I believe button" we proceed to compare $-2 log(\Lambda)$ to a $chi^2(1)$ distribution (1 degree of freedom) and blindly assert that assuming a p value of .05, that our value must be in the 95th percentile or greater to reject the null. But if the null were not true, then we must re-evaluate the statement that it follows a $chi^2(1)$ distribution. We proceed into the unknown, unknowingly....

In [25]:
print('The chi^2 distribution with the 95th percentile:',stats.chi2.ppf(.95,1))
print('Our -2log(Lambda):',-2*log(Cap_Lambda))

The chi^2 distribution with the 95th percentile: 3.841458820694124
Our -2log(Lambda): 1.421222625393962


In conclusion, we see that our likelihood ratio test is less than the 95th percentile of a chi^2 distribution with 1 degree of freedom and questionably conclude that we fail to reject the null hypothesis.

#### Alternative Method

In this method, I simply use the sample mean as the test statistic. The observed value was 1.4. In the below, I simulate under the hypothesized $\lambda$, 1, and determine how often the sample mean was further away from 1 as was 1.4. 

In [21]:
ts2=[np.mean(stats.poisson.rvs(1,size=10)) for _ in np.arange(10000)]
np.mean(ts2>=np.mean(Data))+np.mean(ts2<=(1-(np.mean(Data)-1)))

0.2715

In [27]:
(1-(np.mean(Data)-1))

0.6000000000000001

As we can see here as well, we get a number. That is, 27% of the time the sample mean was further away from 1 as it was 1.4. Cool...? Is 27.15 low? High? Under what values do we reject... and why?